In [1]:
# External imports
import os
import pretty_midi
import numpy as np
import tensorflow as tf
#!apt install libsndfile-dev
#import note_seq
#import IPython

# Internal imports
from data.helpers.midi import MidiEventProcessor
from models.midi_transformer import MIDITransformer

OSError: sndfile library not found

## Required file paths and common variables

In [ ]:
# File paths for local computer
# ------------------------------------------------------------------------------------
#BASE_DIR = "/home/richhiey/Desktop/workspace/projects/virtual_musicians"
#DATA_DIR = os.path.join(BASE_DIR, "data", "POP909-Dataset", "POP909")
#MIDI_EVENTS_PATH = os.path.join(BASE_DIR, "data", "preprocessed",  "pop909-event-token.npy")
#DATASET_PATH = os.path.join(BASE_DIR, "data", "preprocessed",  "pop909.tfrecords")
#MODEL_SAVE_PATH = os.path.join(BASE_DIR, "cache", 'checkpoints', 'model5')
#MODEL_CONFIG_PATH = os.path.join(BASE_DIR, 'model-store', 'models', 'configs', 'default.json')

## File paths for GPU Container
# ------------------------------------------------------------------------------------
BASE_DIR          = "/home/rithomas"
PROJECT_DIR       = os.path.join(BASE_DIR, "project")
DATA_DIR          = os.path.join(BASE_DIR, "data", "POP909-Dataset", "POP909")
MIDI_EVENTS_PATH  = os.path.join(BASE_DIR, "data", "preprocessed",  "pop909-event-token.npy")
DATASET_PATH      = os.path.join(BASE_DIR, "cache", "preprocessed",  "pop909.tfrecords")
MODEL_SAVE_PATH   = os.path.join(BASE_DIR, "cache", 'checkpoints', 'xl-decoder_13')#
MODEL_CONFIG_PATH = os.path.join(PROJECT_DIR, 'model-store', 'models', 'configs', 'default.json')

# Common variables
# ------------------------------------------------------------------------------------
feature_description = {
    'melody': tf.io.VarLenFeature(tf.int64),
    'rhythm': tf.io.VarLenFeature(tf.int64),
    'bridge': tf.io.VarLenFeature(tf.int64)
}
BATCH_SIZE = 16

## Loading POP909 MIDI Dataset

In [ ]:
pop909 = np.load(MIDI_EVENTS_PATH, allow_pickle=True)
print(np.shape(pop909))

melodies = [song['MELODY'] for song in pop909]
rhythms = [song['PIANO'] for song in pop909]
bridges = [song['BRIDGE'] for song in pop909]

if not os.path.exists(DATASET_PATH):
    with tf.io.TFRecordWriter(DATASET_PATH) as file_writer:
        for melody, rhythm, bridge in zip(melodies, rhythms, bridges):
            example = tf.train.Example(
                features=tf.train.Features(
                    feature={
                        "melody": tf.train.Feature(int64_list=tf.train.Int64List(value=[389]+melody+[390])),
                        "rhythm": tf.train.Feature(int64_list=tf.train.Int64List(value=[389]+rhythm+[390])),
                        "bridge": tf.train.Feature(int64_list=tf.train.Int64List(value=[389]+bridge+[390]))
                    }
                )
            )
            file_writer.write(example.SerializeToString())
        file_writer.close()


def _parse_function(example_proto):
    return tf.io.parse_single_example(example_proto, feature_description)

raw_dataset = tf.data.TFRecordDataset(DATASET_PATH)
dataset = raw_dataset.map(_parse_function).batch(BATCH_SIZE, drop_remainder=True).repeat(1000)
event_processor = MidiEventProcessor()
piano = pretty_midi.instrument_name_to_program('Acoustic Grand Piano')

# Reconstruct MIDI data from TFRecord
#for i, data in enumerate(dataset.take(3)):
#    print('----------------------------------------------------------')
#    full_midi = pretty_midi.PrettyMIDI()
    
#    melody_instr = pretty_midi.Instrument(program=piano)
#    rhythm_instr = pretty_midi.Instrument(program=piano)
#    bridge_instr = pretty_midi.Instrument(program=piano)
    
#    print('Melody:')
#    melody_events = tf.sparse.to_dense(data['melody']).numpy()
#    print(melody_events)
#    for note in event_processor.decode(melody_events):
#        melody_instr.notes.append(note)
    
#    print('Rhythm:')
#    rhythm_events = tf.sparse.to_dense(data['rhythm']).numpy()
#    print(rhythm_events)
#    for note in event_processor.decode(rhythm_events):
#        rhythm_instr.notes.append(note)
    
#    print('Bridge:')
#    bridge_events = tf.sparse.to_dense(data['bridge']).numpy()
#    print(bridge_events)
#    for note in event_processor.decode(bridge_events):
#        bridge_instr.notes.append(note)
    
#    full_midi.instruments.append(melody_instr)
#    full_midi.instruments.append(rhythm_instr)
#    full_midi.instruments.append(bridge_instr)
#    IPython.display.display(IPython.display.Audio(full_midi.fluidsynth(), rate=44100))
#    filename = 'test_'+str(i)+'.mid'
#    full_midi.write(filename)
    
#    full_midi_ns = note_seq.midi_io.midi_file_to_note_sequence(filename)
#    note_seq.plot_sequence(full_midi_ns)
#    break

## Train Transformer XL Model

In [ ]:
midi_transformer = MIDITransformer(MODEL_CONFIG_PATH, MODEL_SAVE_PATH)
midi_transformer.reset_states()
event_processor = MidiEventProcessor()

print(midi_transformer)
print(dataset)
midi_transformer.train(dataset, event_processor, {'inputs': 'melody', 'outputs': 'melody', 'num_epochs': '1000'})

## Create new Note Sequences with the MIDI Transformer

In [ ]:
#midi_transformer.predict(midi_note_sequence)